In [1]:
from pyspark.sql import SparkSession

In [2]:
spark_session = SparkSession.builder \
    .master("spark://192.168.2.47:7077") \
    .appName("Create Random Partitions") \
    .config("spark.dynamicAllocation.enabled", True) \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", True) \
    .config("spark.shuffle.service.enabled", False) \
    .config("spark.dynamicAllocation.executorIdleTimeout", "30s") \
    .config("spark.executor.memory", "5G") \
    .config("spark.cores.max", 12) \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/03/19 13:27:45 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/03/19 13:27:46 WARN StandaloneSchedulerBackend: Dynamic allocation enabled without spark.executor.cores explicitly set, you may get more executors allocated than expected. It's recommended to set spark.executor.cores explicitly. Please check SPARK-30299 for more details.


In [3]:
# Loading the dataset without the corrupted values
df = spark_session.read.option("mode", "DROPMALFORMED").json("hdfs://192.168.2.47:9000/data-project/corpus-webis-tldr-17.json")

In [4]:
df.count()

3848330

In [5]:
# Define the number of entries for each partition
partition_sizes = {
    "hdfs://192.168.2.47:9000/data-project/reddit_50k.json": 50000,
    "hdfs://192.168.2.47:9000/data-project/reddit_100k.json": 100000,
    "hdfs://192.168.2.47:9000/data-project/reddit_200k.json": 200000,
    "hdfs://192.168.2.47:9000/data-project/reddit_500k.json": 500000,
}

In [6]:
# Create and save partitions
for output_path, num_entries in partition_sizes.items():
    # Randomly sample the dataset
    fraction = num_entries / df.count()  # Calculate the fraction to sample
    sampled_df = df.sample(withReplacement=False, fraction=fraction, seed=42).limit(num_entries)

    # Save the sampled data to HDFS
    sampled_df.write.mode("overwrite").json(output_path)